Installing Required Packages

This code block uses pip install to install necessary Python libraries such as rdkit, pubchempy, admet_ai, thermo, and setuptools_rust. These packages are essential for cheminformatics tasks, PubChem API interaction, ADMET prediction, and thermochemical property calculations that will be performed later.

In [ ]:
#%%capture
!pip install rdkit
!pip install pubchempy
!pip install admet_ai
!pip install thermo
!pip install setuptools_rust

Loading Required Libraries and Helper Functions

This block imports all the Python libraries required for the workflow, including os, time, logging, concurrent.futures, pandas, tqdm, rdkit, pubchempy, ADMETModel from admet_ai, and Joback from thermo. It also defines several helper functions:

    get_matched_substructures_from_library: Identifies substructures from a library that match a query SMILES.
    generate_transformation_product: Generates reaction products based on a reaction template and query SMILES.
    generate_transformation_products_parallel: Parallelizes the product generation process for efficiency.
    standardize_smiles_safe: Standardizes SMILES strings for consistent chemical representation.
    get_cid_from_smiles_with_retry: Retrieves PubChem CIDs for SMILES with retry logic and caching.
    compute_gibbs_free_energy: Calculates Gibbs free energy for a given SMILES.
    run_ista_workflow: The main function that orchestrates the entire workflow, from matching substructures to predicting ADME properties and Gibbs free energy.

In [ ]:
import os
import time
import logging
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Optional
import pandas as pd
from tqdm.auto import tqdm
from rdkit import Chem
from rdkit.Chem import rdChemReactions, DataStructs
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.warning')
RDLogger.DisableLog('rdApp.info')
import pubchempy as pcp
from admet_ai import ADMETModel
from thermo import Joback


# ----------------- HELPERS -----------------
def get_matched_substructures_from_library(query_smiles: str, sub_struc_lib_df: pd.DataFrame, logger: logging.Logger) -> Optional[pd.DataFrame]:
    if 'Fragmants Smarts' not in sub_struc_lib_df.columns:
        logger.error("Input DataFrame must contain a 'Fragmants Smarts' column.")
        return None
    query_molecule = Chem.MolFromSmiles(query_smiles)
    if query_molecule is None:
        logger.error(f"Invalid query SMILES string: '{query_smiles}'")
        return None

    smarts_patterns = sub_struc_lib_df['Fragmants Smarts'].apply(lambda x: str(x).strip('()')).tolist()
    custom_fp = DataStructs.ExplicitBitVect(len(smarts_patterns))

    for i, smarts in enumerate(smarts_patterns):
        try:
            pattern = Chem.MolFromSmarts(smarts)
            if pattern is None:
                continue
            if query_molecule.HasSubstructMatch(pattern):
                custom_fp.SetBit(i)
        except Exception as e:
            logger.error(f"SMARTS error at index {i}: {e}", exc_info=True)

    matched_indices = [i for i, bit_value in enumerate(custom_fp.ToList()) if bit_value == 1]
    if not matched_indices:
        return None

    matched = sub_struc_lib_df.iloc[matched_indices].copy()
    matched.rename(columns={'Smiles': 'matched_smiles'}, inplace=True)
    matched['query_smiles'] = query_smiles
    return matched


def generate_transformation_product(trans_info: Dict[str, Any]) -> List[Dict[str, Any]]:
    results: List[Dict[str, Any]] = []
    try:
        query_smiles = trans_info['query_smiles']
        match_smiles = trans_info['matched_smiles']
        reac_template = trans_info['reac_temp']
        reaction_smiles = trans_info['SANITIZED_MAPPED_REACTION']
        rxn_id = trans_info['rxn_id']

        updated_reactants = [
            query_smiles if r == match_smiles else r
            for r in reaction_smiles.split('>>')[0].split('.')
        ]
        reactant_mols = [Chem.MolFromSmiles(r) for r in updated_reactants]
        if not all(reactant_mols):
            return []

        reaction = rdChemReactions.ReactionFromSmarts(reac_template)
        if not reaction:
            return []

        products = reaction.RunReactants(reactant_mols)
        unique_smiles_tuples = list({
            tuple(Chem.MolToSmiles(mol, canonical=True) for mol in product_set if mol)
            for product_set in products
        })

        for product_tuple in unique_smiles_tuples:
            results.append({
                'rxn_id': rxn_id,
                'query_smiles': query_smiles,
                'matched_smiles': match_smiles,
                'sanitized_reaction': reaction_smiles,
                'reaction_template': reac_template,
                'transformed_products': product_tuple,
                'rxn_smiles_with_query_cmpd': f"{'.'.join(updated_reactants)}>>{'.'.join(product_tuple)}"
            })
    except Exception:
        return []
    return results


def generate_transformation_products_parallel(merged_info: pd.DataFrame, max_workers: int = 16) -> pd.DataFrame:
    all_results: List[List[Dict[str, Any]]] = []
    rows = merged_info.to_dict(orient="records")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(generate_transformation_product, row) for row in rows]
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing reactions"):
            result = future.result()
            if result:
                all_results.append(result)
    return pd.DataFrame([item for sublist in all_results for item in sublist])


def standardize_smiles_safe(smiles: str) -> Optional[str]:
    from rdkit.Chem.MolStandardize import rdMolStandardize
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        for atom in mol.GetAtoms():
            atom.SetAtomMapNum(0)
        mol = rdMolStandardize.Uncharger().uncharge(mol)
        return Chem.MolToSmiles(mol, canonical=True)
    except Exception:
        return None


def get_cid_from_smiles_with_retry(smiles: str, max_retries: int = 5, initial_delay: float = 1.0, cache: Optional[Dict[str, Optional[str]]] = None) -> Optional[str]:
    if smiles is None:
        return None
    if cache is not None and smiles in cache:
        return cache[smiles]
    delay = initial_delay
    for attempt in range(max_retries):
        try:
            compounds = pcp.get_compounds(smiles, 'smiles')
            if compounds:
                cid = str(compounds[0].cid)
                if cache is not None:
                    cache[smiles] = cid
                return cid
            if cache is not None:
                cache[smiles] = None
            return None
        except pcp.PubChemHTTPError as e:
            if 'ServerBusy' in str(e) or 'PUGREST.ServerBusy' in str(e):
                if attempt < max_retries - 1:
                    time.sleep(delay); delay *= 2
                else:
                    if cache is not None:
                        cache[smiles] = None
                    return None
            else:
                if cache is not None:
                    cache[smiles] = None
                return None
        except Exception:
            if cache is not None:
                cache[smiles] = None
            return None
    return None


def compute_gibbs_free_energy(smiles: str, temp_k: float = 298.15):
    if smiles is None or str(smiles).strip() == '':
        return 'Invalid SMILES'
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return 'Failed To Generate Rdkit Mol Object From Smiles'
    try:
        result = Joback(mol).estimate()
        enthalpy = result['Hf'] / 1000
        gibbs = result['Gf'] / 1000
        entropy = (enthalpy - gibbs) / temp_k
        stability = "Highly Stable" if gibbs < -100 else "Stable" if gibbs < 0 else "Unstable / Highly Reactive"
        return {'Enthalpy': round(enthalpy, 2), 'Entropy': round(entropy, 4), 'Gibbs Free Energy': round(gibbs, 2), 'Stability': stability}
    except Exception as e:
        return {'Enthalpy': None, 'Entropy': None, 'Gibbs Free Energy': None, 'Stability': f'Error: {str(e)}'}


# ----------------- MAIN WORKFLOW -----------------
def run_ista_workflow(
    query_smiles_list: List[str],
    output_dir: str,
    substructure_lib_path: str,
    info_df_path: str,
    log_file: str,
):
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=[logging.FileHandler(log_file), logging.StreamHandler()]
    )
    logger = logging.getLogger(__name__)

    os.makedirs(output_dir, exist_ok=True)
    sub_struc_lib = pd.read_csv(substructure_lib_path)
    info_df = pd.read_csv(info_df_path)

    model = ADMETModel()
    cid_cache: Dict[str, Optional[str]] = {}

    for index, query_smiles in enumerate(tqdm(query_smiles_list, desc="Query compounds"), start=1):
        matched_substruc_df = get_matched_substructures_from_library(query_smiles, sub_struc_lib, logger)
        if matched_substruc_df is None or matched_substruc_df.empty:
            continue

        merged_info = pd.merge(
            matched_substruc_df,
            info_df[['RXN_ID', 'SANITIZED_MAPPED_REACTION', 'reac_temp']],
            left_on='rxn_id',
            right_on='RXN_ID',
            how='inner'
        ).drop(columns=['RXN_ID'])

        result = pd.DataFrame(generate_transformation_products_parallel(merged_info, max_workers=16))
        if result.empty:
            continue

        # Add standardized transformed products
        result["standardized_transformed_products"] = result["transformed_products"].apply(
            lambda tup: [s for s in (standardize_smiles_safe(x) for x in tup) if s]
        )

        subdir = os.path.join(output_dir, f'query_cmpd_{index}')
        os.makedirs(subdir, exist_ok=True)
        result.to_csv(os.path.join(subdir, f'query_compound_{index}_transformation_products.csv'), index=False)

        # ADME table (products only)
        unique_product_smiles = sorted({p for tup in result["standardized_transformed_products"] for p in tup if p})
        adme_prop = []
        for smi in unique_product_smiles:
            try:
                pred = model.predict(smi)
                pred['smiles'] = smi
                adme_prop.append(pred)
            except Exception as e:
                logger.error(f"Error predicting ADME properties for SMILES {smi}: {e}")

        adme_prop_df = pd.DataFrame(adme_prop)
        adme_prop_df['Parent Compound/Product'] = 'Product'
        adme_prop_query = pd.DataFrame([{**model.predict(query_smiles), 'smiles': query_smiles, 'Parent Compound/Product': 'Parent Compound'}])
        adme_table = pd.concat([adme_prop_query, adme_prop_df], ignore_index=True)

        column_order = ['Parent Compound/Product', 'smiles']
        remaining_cols = [col for col in adme_table.columns if col not in column_order]
        adme_table = adme_table[column_order + remaining_cols]

        adme_table.to_csv(os.path.join(subdir, f'adme_properties_query_compound_{index}.csv'), index=False)

        # Gibbs free energy (products only)
        gibbs_rows = []
        for smi in unique_product_smiles:
            energy_result = compute_gibbs_free_energy(smi)
            if isinstance(energy_result, dict):
                gibbs_rows.append({"standardized_smiles": smi, **energy_result})
            elif isinstance(energy_result, (int, float)):
                gibbs_rows.append({"standardized_smiles": smi, "Gibbs_Free_Energy": energy_result})
            else:
                gibbs_rows.append({"standardized_smiles": smi, "error": str(energy_result)})

        gibbs_df = pd.DataFrame(gibbs_rows)
        if 'Gibbs_Free_Energy' in gibbs_df.columns:
            gibbs_df['rank'] = gibbs_df['Gibbs_Free_Energy'].rank(ascending=True, method='min')
        elif 'Gibbs Free Energy' in gibbs_df.columns:
            gibbs_df['rank'] = gibbs_df['Gibbs Free Energy'].rank(ascending=True, method='min')

        gibbs_df.to_csv(os.path.join(subdir, f'query_compound_{index}_gibbs_free_energy.csv'), index=False)

        # Combined per-query result table
        std_products_df = pd.DataFrame({"standardized_smiles": unique_product_smiles})

        adme_products = adme_prop_df.copy()
        adme_products = adme_products.rename(columns={"smiles": "standardized_smiles"})

        # add CID for standardized products
        adme_products["cid"] = adme_products["standardized_smiles"].apply(
            lambda s: get_cid_from_smiles_with_retry(s, cache=cid_cache)
        )

        per_query_result = (
            std_products_df
            .merge(adme_products, on="standardized_smiles", how="left")
            .merge(gibbs_df, on="standardized_smiles", how="left")
        )

        per_query_result.to_csv(
            os.path.join(subdir, f"query_compound_{index}_results_table.csv"),
            index=False
        )


Data Preparation and Execution

This block performs initial setup by creating directories for storing results and source files. It then downloads two critical CSV files from Zenodo: Table_2_Chemical_Reaction_Analysis.csv (containing reaction information) and Table_3_Substructure_Library.csv (the library of substructures). These files are fundamental inputs for the run_ista_workflow function.

In [ ]:
os.makedirs('/content/ista_results', exist_ok= True)
os.makedirs('/content/ista_results/source_files', exist_ok= True)

!wget -O /content/ista_results/source_files/Table_2_Chemical_Reaction_Analysis.csv https://zenodo.org/records/17576138/files/Table%202_Chemical%20Reaction%20Analysis.csv?download=1
!wget -O /content/ista_results/source_files/Table_3_Substructure_Library.csv https://zenodo.org/records/17576138/files/Table%203_Substructure%20Library.csv?download=1

 Setting up File Paths

 This block defines the file paths for the log file, output directory, substructure library, and reaction information DataFrame. These variables are used as arguments when calling the run_ista_workflow function, ensuring that all outputs are saved in an organized manner.

In [ ]:
log_file= '/content/ista_results/ista_workflow/log/log_file'
output_dir= '/content/ista_results/ista_workflow/output'
substructure_lib_path= '/content/ista_results/source_files/Table_3_Substructure_Library.csv'
info_df_path= '/content/ista_results/source_files/Table_2_Chemical_Reaction_Analysis.csv'

Running the ISTA Workflow (cell ID: z65Zfy7TK31m)

This final block initializes query_smiles_list with a single SMILES string ('CC(=O)Nc1ccc(O)cc1') which is Acetaminophen, and then calls the main run_ista_workflow function with all the predefined parameters. This initiates the entire process of identifying potential transformations, generating products, and calculating their ADME properties and Gibbs free energies.

In [ ]:
query_smiles_list= ['CC(=O)Nc1ccc(O)cc1']


run_ista_workflow(
    query_smiles_list=query_smiles_list,
    output_dir=output_dir,
    substructure_lib_path=substructure_lib_path,
    info_df_path=info_df_path,
    log_file=log_file,
)